In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/pggan/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/pggan/128/b0/to_star/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'star/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/pggan/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=2, total_epoch=50, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + 0.1*loss_sp + 0.1*loss_cls

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [20]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']

    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()
    
    if (epoch+1)%100 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.1393 | MainLoss:0.8272 | Alpha:0.0586 | SPLoss:1.7272 | CLSLoss:1.3942 | AUROC:0.0000
Test | 122/16 | Loss:0.6933 | MainLoss:0.6933 | SPLoss:1.5800 | CLSLoss:1.1234 | AUROC:0.4871
Test | 128/16 | Loss:0.6823 | MainLoss:0.6823 | SPLoss:1.5800 | CLSLoss:1.1234 | AUROC:0.9989

Epoch: [2 | 1000] LR: 0.102000
Train | 16/16 | Loss:0.9219 | MainLoss:0.6931 | Alpha:0.0539 | SPLoss:1.3364 | CLSLoss:0.9515 | AUROC:0.0000
Test | 122/16 | Loss:0.6933 | MainLoss:0.6933 | SPLoss:1.0932 | CLSLoss:0.7797 | AUROC:0.4970
Test | 128/16 | Loss:0.6645 | MainLoss:0.6645 | SPLoss:1.0932 | CLSLoss:0.7797 | AUROC:0.9998

Epoch: [3 | 1000] LR: 0.104000
Train | 16/16 | Loss:0.8489 | MainLoss:0.6912 | Alpha:0.0563 | SPLoss:0.9175 | CLSLoss:0.6598 | AUROC:0.0000
Test | 122/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.7431 | CLSLoss:0.5398 | AUROC:0.5100
Test | 128/16 | Loss:0.6225 | MainLoss:0.6225 | SPLoss:0.7431 | CLSLoss:0.5398 | AUROC:1.0000

Epoch: [4 | 

Train | 16/16 | Loss:190.6612 | MainLoss:0.5822 | Alpha:0.0559 | SPLoss:1900.7179 | CLSLoss:0.0722 | AUROC:0.0000
Test | 122/16 | Loss:0.5504 | MainLoss:0.5504 | SPLoss:1411.3712 | CLSLoss:0.0627 | AUROC:0.8228
Test | 128/16 | Loss:0.2438 | MainLoss:0.2438 | SPLoss:1411.3713 | CLSLoss:0.0627 | AUROC:0.9978

Epoch: [27 | 1000] LR: 0.152000
Train | 16/16 | Loss:114.0621 | MainLoss:0.5731 | Alpha:0.0585 | SPLoss:1134.8213 | CLSLoss:0.0690 | AUROC:0.0000
Test | 122/16 | Loss:0.5616 | MainLoss:0.5616 | SPLoss:849.2826 | CLSLoss:0.0847 | AUROC:0.8268
Test | 128/16 | Loss:0.2685 | MainLoss:0.2685 | SPLoss:849.2824 | CLSLoss:0.0847 | AUROC:0.9975

Epoch: [28 | 1000] LR: 0.154000
Train | 16/16 | Loss:66.9235 | MainLoss:0.5344 | Alpha:0.0549 | SPLoss:663.8080 | CLSLoss:0.0822 | AUROC:0.0000
Test | 122/16 | Loss:0.5444 | MainLoss:0.5444 | SPLoss:488.7552 | CLSLoss:0.0938 | AUROC:0.8398
Test | 128/16 | Loss:0.2979 | MainLoss:0.2979 | SPLoss:488.7552 | CLSLoss:0.0938 | AUROC:0.9958

Epoch: [29 | 10

Test | 128/16 | Loss:0.2836 | MainLoss:0.2836 | SPLoss:3.3750 | CLSLoss:0.0466 | AUROC:0.9989

Epoch: [51 | 1000] LR: 0.200000
Train | 16/16 | Loss:0.9598 | MainLoss:0.5536 | Alpha:0.0546 | SPLoss:3.9915 | CLSLoss:0.0709 | AUROC:0.0000
Test | 122/16 | Loss:0.5106 | MainLoss:0.5106 | SPLoss:11.0983 | CLSLoss:0.0883 | AUROC:0.8431
Test | 128/16 | Loss:0.1325 | MainLoss:0.1325 | SPLoss:11.0983 | CLSLoss:0.0883 | AUROC:0.9996

Epoch: [52 | 1000] LR: 0.200000
Train | 16/16 | Loss:1.4454 | MainLoss:0.5942 | Alpha:0.0583 | SPLoss:8.4461 | CLSLoss:0.0666 | AUROC:0.0000
Test | 122/16 | Loss:0.5293 | MainLoss:0.5293 | SPLoss:5.8809 | CLSLoss:0.0825 | AUROC:0.8351
Test | 128/16 | Loss:0.2255 | MainLoss:0.2255 | SPLoss:5.8809 | CLSLoss:0.0825 | AUROC:0.9994

Epoch: [53 | 1000] LR: 0.200000
Train | 16/16 | Loss:1.0581 | MainLoss:0.5534 | Alpha:0.0579 | SPLoss:4.9526 | CLSLoss:0.0940 | AUROC:0.0000
Test | 122/16 | Loss:0.5324 | MainLoss:0.5324 | SPLoss:4.2228 | CLSLoss:0.0588 | AUROC:0.8371
Test | 1

Test | 128/16 | Loss:0.2721 | MainLoss:0.2721 | SPLoss:4.8091 | CLSLoss:0.0617 | AUROC:0.9996

Epoch: [76 | 1000] LR: 0.199716
Train | 16/16 | Loss:1.2855 | MainLoss:0.6017 | Alpha:0.0561 | SPLoss:6.7678 | CLSLoss:0.0704 | AUROC:0.0000
Test | 122/16 | Loss:0.6172 | MainLoss:0.6172 | SPLoss:8.1422 | CLSLoss:0.0304 | AUROC:0.7924
Test | 128/16 | Loss:0.4243 | MainLoss:0.4243 | SPLoss:8.1422 | CLSLoss:0.0304 | AUROC:0.9985

Epoch: [77 | 1000] LR: 0.199692
Train | 16/16 | Loss:1.2114 | MainLoss:0.5860 | Alpha:0.0565 | SPLoss:6.1851 | CLSLoss:0.0687 | AUROC:0.0000
Test | 122/16 | Loss:0.6317 | MainLoss:0.6317 | SPLoss:4.5992 | CLSLoss:0.0347 | AUROC:0.7654
Test | 128/16 | Loss:0.3512 | MainLoss:0.3512 | SPLoss:4.5992 | CLSLoss:0.0347 | AUROC:0.9996

Epoch: [78 | 1000] LR: 0.199667
Train | 16/16 | Loss:0.9639 | MainLoss:0.5888 | Alpha:0.0571 | SPLoss:3.6839 | CLSLoss:0.0672 | AUROC:0.0000
Test | 122/16 | Loss:0.5715 | MainLoss:0.5715 | SPLoss:3.1963 | CLSLoss:0.0601 | AUROC:0.8016
Test | 128

Test | 128/16 | Loss:0.2905 | MainLoss:0.2905 | SPLoss:2.5719 | CLSLoss:0.0471 | AUROC:0.9994

Epoch: [101 | 1000] LR: 0.198817
Train | 16/16 | Loss:0.5603 | MainLoss:0.5436 | Alpha:0.3631 | SPLoss:0.0816 | CLSLoss:0.0854 | AUROC:0.0000
Test | 122/16 | Loss:0.4975 | MainLoss:0.4975 | SPLoss:0.1508 | CLSLoss:0.0836 | AUROC:0.8552
Test | 128/16 | Loss:0.2205 | MainLoss:0.2205 | SPLoss:0.1508 | CLSLoss:0.0836 | AUROC:0.9991

Epoch: [102 | 1000] LR: 0.198769
Train | 16/16 | Loss:0.5438 | MainLoss:0.5127 | Alpha:0.3634 | SPLoss:0.2079 | CLSLoss:0.1029 | AUROC:0.0000
Test | 122/16 | Loss:0.5219 | MainLoss:0.5219 | SPLoss:0.2646 | CLSLoss:0.0741 | AUROC:0.8624
Test | 128/16 | Loss:0.2173 | MainLoss:0.2173 | SPLoss:0.2646 | CLSLoss:0.0741 | AUROC:0.9994

Epoch: [103 | 1000] LR: 0.198719
Train | 16/16 | Loss:0.5592 | MainLoss:0.5184 | Alpha:0.3617 | SPLoss:0.3061 | CLSLoss:0.1024 | AUROC:0.0000
Test | 122/16 | Loss:0.6464 | MainLoss:0.6464 | SPLoss:0.3825 | CLSLoss:0.0804 | AUROC:0.8594
Test | 

...

Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [901 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.1920 | MainLoss:0.1778 | Alpha:0.4591 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9266
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9991

Epoch: [902 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.1981 | MainLoss:0.1839 | Alpha:0.4582 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9266
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [903 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.1787 | MainLoss:0.1645 | Alpha:0.4617 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9266
Test | 

Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [926 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.1762 | MainLoss:0.1620 | Alpha:0.4628 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9268
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [927 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.1943 | MainLoss:0.1801 | Alpha:0.4581 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9267
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [928 | 1000] LR: 0.000001
Train | 16/16 | Loss:0.1663 | MainLoss:0.1521 | Alpha:0.4642 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9269
Test | 

Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [951 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.1889 | MainLoss:0.1747 | Alpha:0.4600 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9265
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [952 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.1833 | MainLoss:0.1691 | Alpha:0.4617 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9267
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [953 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.1971 | MainLoss:0.1829 | Alpha:0.4566 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9264
Test | 

Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [976 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.2026 | MainLoss:0.1884 | Alpha:0.4562 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9268
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [977 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.1794 | MainLoss:0.1652 | Alpha:0.4620 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9269
Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990

Epoch: [978 | 1000] LR: 0.000000
Train | 16/16 | Loss:0.1762 | MainLoss:0.1620 | Alpha:0.4625 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.0000
Test | 122/16 | Loss:0.3931 | MainLoss:0.3931 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9266
Test | 

Test | 128/16 | Loss:0.0662 | MainLoss:0.0662 | SPLoss:0.0000 | CLSLoss:0.1418 | AUROC:0.9990
